# Analitzador de codi

----
<small><i>Gener 2024 - *Notebook creat per Lluís Santacreu</i></small>

##Objectius
 - Entendre el funcionament del model *chat openai* per analitzar codi.
 - Aprendre a utilitzar la Generació Augmentada de Recuperació, RAG.

 En aquesta pràctica s'utilitzarà la RAG per tal de respondre a preguntes que tenen com a objectiu analitzar al codi en Python d'una aplicació que haurem posat en un repositori.

 Haurem de tenir una openAI_key. Cal anar a openai.com, registrar-se i accedir a la seva API. Caldrà fer un petit pagament de 5€ com a dipòsit (es recomana utilitzar una tarja bancària de despesa limitada o, si és possible, un número de tarjeta d'un sol ús). Anar a les propietats del nostre usuari i a l'opció API keys, crear-ne una amb el botó *Create new secret key*. Li posem un nom i la desem en un lloc segur (al nostre ordinador o al Drive). El cost de cada execució és d'uns 0,001$.


In [ ]:
# Importar dades des del Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


## Instal·lació de requeriments
- La llibreria **openai** proveeix accès a l'API OpenaAI REST. Inclou definicions per als paràmetres de les preguntes i camps de les repostes.
- La llibreria **tiktoken** permet transformar les paraules del text en tokens.
- La llibreria **chromadb** és l'encarregada de fer les incrustacions (embeddings).
- La llibreria **langchain** permet enllaçar les dades amb el model.
- La llibreria **gitpython** permet interactuar amb els repositoris de github.


In [ ]:
!pip install openai tiktoken chromadb langchain gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings.openai import OpenAIEmbeddings


##Obtenció del codi a analitzar
Es crea un diretori /content/portfolio/ on descarreguem una aplicació del github que es desitgi analitzar. En aquest cas els d'una aplicació que serveix per construir portafolis (carteres amb informes de clients).

Després es carreguen a la memòria els fitxers .py .

In [ ]:
repo_path = '/content/drive/MyDrive/IOC/IABD/M1/EAC4'

In [ ]:
#De l'aplicació, es seleccionen només els fitxers .py
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents=loader.load()
#es compten el nombre de fitxers carregats
len(documents)

1

##Preprocessament del codi
S'utilitza RecursiveCaracterTextSplitter per dividir el text del codi en segments. Aquestes segments seran de 2000 caràcters i es solaparan 200caracters entre ells per tal que el model els pugui relacionar.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=2000,
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)

In [ ]:
texts

[Document(page_content="import os\nimport random\nimport string\n\n# Define the length of the random string\nlength = 10\n\n# Generate a string of random bytes\nrandom_bytes = os.urandom(length)\n\n# Convert the random bytes to a string\nstring_list = []\nfor i in range(10):\n    random_string = ''.join([random.choice(string.ascii_letters + string.digits) for n in range(length)])\n    string_list.append(random_string)\n\nprint(string_list)", metadata={'source': '/content/drive/MyDrive/IOC/IABD/M1/EAC4/code.py', 'language': <Language.PYTHON: 'python'>})]

## Connexió amb el LLM

L'LLM crea les incrustacions (embeddings), es necessària l'OpenAI_Key.

Chroma determina la similaritat amb les preguntes que se li passaran.

El model concret serà el *gpt-3.5-turbo*.


In [ ]:
import os
os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1/"
os.environ["OPENAI_API_KEY"] = ""
embeding = OpenAIEmbeddings()
model_name="gpt-3.5-turbo"

In [ ]:
#Es defineix la clau personal d'OpenaAI
#Creació d'incrustacions
db = Chroma.from_documents(texts,embeding )
# Paràmetres de similaritat
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8},
)

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
qa=ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

## Realització de preguntes i obtenció de les respostes

L'anàlisi del codi es fa passant preguntes al model i aquest les respon.

In [ ]:
#Una pregunta aïllada
question = "Com puc trobar el valor mínim de l'array numeros al document code.py?"
result= qa(question)
result['answer']

'Per trobar el valor mínim de l\'array "numeros" en el document code.py, primer caldria assegurar-se que l\'array "numeros" estigui definit i que es pugui accedir a ell des del codi del document code.py. A partir d\'aquí, pots utilitzar la funció `min()` de Python per trobar el valor mínim d\'aquest array. Si l\'array "numeros" no està definit en el codi proporcionat, no es pot determinar el valor mínim.'

# Ollama test

Objectiu, utilitzar models oberts desde colab hospedats desde el meu equip.


In [ ]:
# Importar dades des del Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


## Instal·lació de requeriments
- La llibreria **openai** proveeix accès a l'API OpenaAI REST. Inclou definicions per als paràmetres de les preguntes i camps de les repostes.
- La llibreria **tiktoken** permet transformar les paraules del text en tokens.
- La llibreria **chromadb** és l'encarregada de fer les incrustacions (embeddings).
- La llibreria **langchain** permet enllaçar les dades amb el model.
- La llibreria **gitpython** permet interactuar amb els repositoris de github.


In [ ]:
!pip install openai tiktoken chromadb langchain gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings.openai import OpenAIEmbeddings


##Obtenció del codi a analitzar
Es crea un diretori /content/portfolio/ on descarreguem una aplicació del github que es desitgi analitzar. En aquest cas els d'una aplicació que serveix per construir portafolis (carteres amb informes de clients).

Després es carreguen a la memòria els fitxers .py .

In [ ]:
repo_path = '/content/drive/MyDrive/IOC/IABD/M1/EAC4'

In [ ]:
#De l'aplicació, es seleccionen només els fitxers .py
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents=loader.load()
#es compten el nombre de fitxers carregats
len(documents)

1

##Preprocessament del codi
S'utilitza RecursiveCaracterTextSplitter per dividir el text del codi en segments. Aquestes segments seran de 2000 caràcters i es solaparan 200caracters entre ells per tal que el model els pugui relacionar.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                               chunk_size=2000,
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)

In [ ]:
texts

[Document(page_content="import os\nimport random\nimport string\n\n# Define the length of the random string\nlength = 10\n\n# Generate a string of random bytes\nrandom_bytes = os.urandom(length)\n\n# Convert the random bytes to a string\nstring_list = []\nfor i in range(10):\n    random_string = ''.join([random.choice(string.ascii_letters + string.digits) for n in range(length)])\n    string_list.append(random_string)\n\nprint(string_list)", metadata={'source': '/content/drive/MyDrive/IOC/IABD/M1/EAC4/code.py', 'language': <Language.PYTHON: 'python'>})]

## Connexió amb el LLM

L'LLM crea les incrustacions (embeddings), es necessària l'OpenAI_Key.

Chroma determina la similaritat amb les preguntes que se li passaran.

El model concret serà el *gpt-3.5-turbo*.


In [ ]:
import os
os.environ["OPENAI_API_BASE"] = "https://llama3.gotu.com/v1/"
os.environ["OPENAI_API_KEY"] = "EMPTY"
embeding = OllamaEmbeddings(model="all-minilm", base_url="https://embedings.gotu.com")
model_name="llama3"

In [ ]:
#Es defineix la clau personal d'OpenaAI
#Creació d'incrustacions
db = Chroma.from_documents(texts,embeding )
# Paràmetres de similaritat
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8},
)

In [ ]:
llm = ChatOpenAI(model_name=model_name)
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
qa=ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

## Realització de preguntes i obtenció de les respostes

L'anàlisi del codi es fa passant preguntes al model i aquest les respon.

In [ ]:
#Una pregunta aïllada
question = "Com puc trobar el valor mínim de l'array numeros al document code.py?"
result= qa(question)
result['answer']

'Per trobar el valor mínim de l\'array "numeros" en el document code.py, primer caldria assegurar-se que l\'array "numeros" estigui definit i que es pugui accedir a ell des del codi del document code.py. A partir d\'aquí, pots utilitzar la funció `min()` de Python per trobar el valor mínim d\'aquest array. Si l\'array "numeros" no està definit en el codi proporcionat, no es pot determinar el valor mínim.'